# Models with Modular Data Pipelines

In [ ]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("../../src")

In [ ]:
import sensai
import sensai.xgboost
import sensai.torch
import pandas as pd
import torch

## VectorModel

The backbone of supervised learning implementations is the `VectorModel` abstraction.
It is so named, because, in computer science, a *vector* corresponds to an array of data,
and vector models map such vectors to the desired outputs, i.e. regression targets or 
classes.

It is important to note that this does *not* limit vector models to tabular data, because the data within
a vector can take arbitrary forms (in contrast to vectors as they are defined in mathematics).
Every element of an input vector could itself be arbitrary
complex, and could, in the most general sense, be any kind of object.

### The VectorModel Class Hierarchy

`VectorModel` is an abstract base class.
From it, abstract base classes for classification (`VectorClassificationModel`) and regression (`VectorRegressionModel`) are derived. And we furthermore provide base classes for rule-based models, facilitating the implementation of models that do not require learning (`RuleBasedVectorClassificationModel`, `RuleBasedVectorRegressionModel`).

These base classes are, in turn, specialised in order to provide direct access to model implementations based on widely used machine learning libraries such as scikit-learn, XGBoost, PyTorch, etc.
Use your IDE's hierarchy view to inspect them.

<!-- TODO: hierarchical bullet item list with hierarchy (or maybe auto-generate?) -->

### DataFrame-Based Interfaces

Vector models use pandas DataFrames as the fundmental input and output data structures.
Every row in a data frame corresponds to a vector of data, and an entire data frame can thus be viewed as a dataset or batch of data. Data frames are a good base representation for input data because
  * they provide rudimentary meta-data in the form of column names, avoiding ambiguity.
  * they can contain arbitrarily complex data, yet in the simplest of cases, they can directly be mapped to a data matrix (2D array) of features that simple models can directly process.

The `fit` and `predict` methods of `VectorModel` take data frames as input, and the latter furthermore returns its predictions as a data frame.
It is important to note that the DataFrame-based interface does not limit the scope of the models that can be applied, as one of the key principles of vector models is that they may define arbitrary model-specific transformations of the data originally contained in a data frame (e.g. a conversion from complex objects in data frames to one or more tensors for neural networks), as we shall see below.

Here's the particularly simple Iris dataset for flower species classification, where the features are measurements of petals and sepals:

In [ ]:
dataset = sensai.data.dataset.DataSetClassificationIris()
io_data = dataset.load_io_data()
io_data.to_df().sample(8)

Here, `io_data` is an instance of `InputOutputData`, which contains two data frames, `inputs` and `outputs`.
The `to_df` method merges the two data frames into one for easier visualisation.

Let's split the dataset and apply a model to it:

In [ ]:
# load and split a dataset
splitter = sensai.data.DataSplitterFractional(0.8)
train_io_data, test_io_data = splitter.split(io_data)

# train a model
model = sensai.sklearn.classification.SkLearnRandomForestVectorClassificationModel(
    n_estimators=15)
model.fit_input_output_data(train_io_data)

# make predictions
predictions = model.predict(test_io_data.inputs)

The `fit_input_output_data` method is just a convenience method to pass an `InputOutputData` instance instead of two data frames. It is equivalent to

```python
model.fit(train_io_data.inputs, train_io_data.outputs)
```

where the two data frames containing inputs and outputs are passed separately.

Now let's compare the ground truth to some of the predictions:

In [ ]:
pd.concat((test_io_data.outputs, predictions), axis=1).sample(8)

### Implementing Custom Models

It is straightforward to implement your own model. Simply subclass the appropriate base class depending on the type of model you want to implement.

For example, let us implement a simple classifier where we always return the a priori probability of each class in the training data, ignoring the input data for predictions. For this case, we inherit from `VectorClassificationModel` and implement the two abstract methods it defines.

In [ ]:
class PriorProbabilityVectorClassificationModel(sensai.VectorClassificationModel):
    def _fit_classifier(self, x: pd.DataFrame, y: pd.DataFrame):
        self._prior_probabilities = y.iloc[:, 0].value_counts(normalize=True).to_dict()

    def _predict_class_probabilities(self, x: pd.DataFrame) -> pd.DataFrame:
        values = [self._prior_probabilities[cls] for cls in self.get_class_labels()]
        return pd.DataFrame([values] * len(x), columns=self.get_class_labels(), index=x.index)

Adapting a model implementation from another machine learning library is typically just a few lines. For models that adhere to the scikit-learn interfaces for learning and prediction, there are abstract base classes that make the adaptation particularly straightforward.

### Configuration

Apart from the parameters passed at construction, which are specific to the type of model in question, all vector models can be flexibly configured via methods that can be called post-construction.
These methods all have the `with_` prefix, indicating that they return the instance itself (akin to the builder pattern), allowing calls to be chained in a single statement.

The most relevant such methods are:

* `with_name` to name the model (for reporting purposes)
* `with_raw_input_transformer` for adding an initial input transformation
* `with_feature_generator` and `with_feature_collector` for specifying how to generate features from the input data
* `with_feature_transformers` for specifying how the generated features shall be transformed

The latter three points are essential for defining modular input pipelines and will be addressed in detail below.

All configured options are fully reflected in the model's string representation, which can be pretty-printed with the `pprint` method.

In [ ]:
str(model.with_name("RandomForest"))

In [ ]:
model.pprint()

## Modular Pipelines

A key principle of sensAI's vector models is that data pipelines 
* can be **strongly associated with a model**. This is critically important of several heterogeneous models shall be applied to the same use case. Typically, every model has different requirements regarding the data it can process and the representation it requires to process it optimally.
* are to be **modular**, meaning that a pipeline can be composed from reusable and user-definable components.

An input pipeline typically serves the purpose of answering the following questions:

* **How shall the data be pre-processed?**

  It might be necessary to process the data before we can use it and extract data from it.
  We may need to filter or clean the data;
  we may need to establish a usable representation from raw data (e.g. convert a string-based representation of a date into a proper data structure);
  or we may need to infer/impute missing data.

  The relevant abstraction for this task is `DataFrameTransformer`, which, as the name suggests, can arbitrarily transform a data frame.
  All non-abstract class implementations have the prefix `DFT` in sensAI and thus are easily discovered through auto-completion.

  A `VectorModel` can be configured to apply a pre-processing transformation via method `with_raw_input_transformers`.

* **What is the data used by the model?**

  The relevant abstraction is `FeatureGenerator`. Via `FeatureGenerator` instances, a model can define which set of features is to be used. Moreover, these instances can hold meta-data on the respective features, which can be leveraged for downstream representation. 
  In sensAI, the class names of all feature generator implementations use the prefix `FeatureGenerator`.

  A `VectorModel` can be configured to answer this question via method `with_feature_generator` (or `with_feature_collector`).

* **How does that data need to be represented?**

  Different models can require different representations of the same data. For example, some models might require all features to be numeric, thus requiring categorical features to be encoded, while others might work better with the original representation.
  Furthermore, some models might work better with numerical features normalised or scaled in a certain way while it makes no difference to others.
  We can address these requirements by adding model-specific transformations.

  The relevant abstraction is, once again, `DataFrameTransformer`.

  A `VectorModel` can be configured to apply a transformation to its features via method `with_feature_transformers`.

The three pipeline stages are applied in the order presented above, and all components are optional, i.e. if a model does not define any raw input transformers, then the original data remains unmodified. If a model defines no feature generator, then the set of features is given by the full input data frame, etc.

### Example Dataset: Titanic Survival

Let us consider the well-known Titanic Survival data set as an example.

Every data point holds data on a passenger. The data set has the following potentially predictive columns,

* `Pclass`: the passenger ticket class as an integer (1=first, 2=second, 3=third)
* `Sex`: the passenger's sex (male or female)
* `Age`: the passenger's age in years (integer); this feature is frequently missing
* `SibSp`: the number of siblings and spouses of the passenger
* `Parch`: the number of parents and children of the passenger
* `Fare`: the fare price paid
* `Embark`: the port of embarkation (C=Cherbourg, Q=Queenstown, S=Southampton); this feature is missing for two passengers

and some further meta-data columns (Name, Cabin).

The goal is to predict the column 'Survived' indicating whether the passenger survived (1) or not (0).

In [ ]:
dataset = sensai.data.dataset.DataSetClassificationTitanicSurvival()
io_data = dataset.load_io_data()
io_data.to_df().iloc[:5]

Let us define an evaluation object for this data set, which will allow us to evaluate model performance later.

In [ ]:
evaluator_params = sensai.evaluation.ClassificationEvaluatorParams(fractional_split_test_fraction=0.2)
titanic_evaluation = sensai.evaluation.ClassificationModelEvaluation(io_data, evaluator_params=evaluator_params)

### Raw Input Transformers

We shall now add pipeline components to an XGBoost model, as it can straightforwardly deal with missing data.

The dataset doesn't really require any pre-processing, but we could 
* get rid of the useless meta-data columns,
* convert the passenger class feature into a string to ensure that it is not treated as a numerical feature

In [ ]:
class DFTTitanicDropMetaDataColumns(sensai.data_transformation.DFTColumnFilter):
    def __init__(self):
        super().__init__(drop=[dataset.COL_NAME, dataset.COL_CABIN, dataset.COL_TICKET])
        
class DFTTitanicTransformPassengerClass(sensai.data_transformation.DFTModifyColumn):
    def __init__(self):
        super().__init__(
            column=dataset.COL_PASSENGER_CLASS, 
            column_transform=lambda n: {1: "first", 2: "second", 3: "third"}[n])

xgb_model = sensai.xgboost.XGBGradientBoostedVectorClassificationModel() \
    .with_raw_input_transformers(
        DFTTitanicDropMetaDataColumns(),
        DFTTitanicTransformPassengerClass())

Our model uses two data frame transformers which apply the aforementioned pre-processing tasks.
We have opted to define classes for each transformation to facilitate reusing the transformations for other models. 

We can apply the transformers using the model's `compute_model_inputs` method.
Since neither transformer requires fitting, we can directly apply it.

In [ ]:
xgb_model.compute_model_inputs(io_data.inputs).iloc[:5]

The model's input pipeline now transforms the data as desired.

### Feature Generation and Transformation

Feature generators serve to define how features can be generated from the input data.
They additionally hold meta-data on the generated features, which can be leveraged for downstream transformation. Specifically,
* we can define which features are categorical,
* we can define rules for normalisation or scaling of numerical features. 

#### Simple Feature Pipelines for Titanic Survival

In the Titanic Survival data set, the features are already fully prepared, so we do not need to actually generate anything; 
we can simply take the feature values as they are present in the original data frame and add only the necessary meta-data.
The base class for this purpose is `FeatureGeneratorTakeColumns`, which allows us to take over columns directly from the input data.
We could use a single feature generator for all features as follows:

In [ ]:
class FeatureGeneratorTitanicAll(sensai.featuregen.FeatureGeneratorTakeColumns):
    def __init__(self):
        super().__init__(
            columns=None,  # take all columns
            categorical_feature_names=[dataset.COL_SEX, dataset.COL_PASSENGER_CLASS, dataset.COL_PORT_EMBARKED],
            normalisation_rule_template=sensai.featuregen.DFTNormalisation.RuleTemplate(
                transformer_factory=sensai.data_transformation.SkLearnTransformerFactoryFactory.StandardScaler(),
                independent_columns=True))

We have supplied both meta-data regarding 
* the subset of feature that are categorical
* the normalisation rule to be applied to the numerical features (if normalisation is applied with `DFTNormalisation`).

Our XGBoost model does not require normalisation, but we still want to apply a transformation to some of the features:
Categorical feature shall be one-hot encoded.
To achieve this, we add the feature generator as well as a DFT that applies the one-hot encoding: 

In [ ]:
feature_generator = FeatureGeneratorTitanicAll()

xgb_model.with_feature_generator(feature_generator) \
    .with_feature_transformers(
        sensai.data_transformation.DFTOneHotEncoder(
            feature_generator.get_categorical_feature_name_regex(), 
            ignore_unknown=True)) \
    .pprint()

When using more than one feature generator, the feature generators need to be combined into a `MultiFeatureGenerator`.
To facilitate this and to furthermore simplify the creation of downstream transformers, we can instead use the `FeatureCollector` abstraction.
The above model can be equivalently defined as follows:

In [ ]:
feature_collector = sensai.featuregen.FeatureCollector(feature_generator)  # can pass more than one feature generator

xgb_model.with_feature_collector(feature_collector, shared=True) \
    .with_feature_transformers(feature_collector.create_feature_transformer_one_hot_encoder(ignore_unknown=True));

Either way, the model's feature pipeline is now fully configured.
The full pipeline now requires fitting (since the feature transformation is learnt from the training data).
Let's fit the model and then take another look at the inputs that the XGBoost model now actually receives.

In [ ]:
xgb_model.fit_input_output_data(io_data)
xgb_model.compute_model_inputs(io_data.inputs).iloc[:5]

The pipeline works as expected. We are now ready to evaluate the model:

In [ ]:
result = titanic_evaluation.perform_simple_evaluation(xgb_model)
result.get_data_frame()

In the demonstration above, the model's full definition was spread out across several cells to incrementally explain the pipeline construction.
In practice we want to keep the model definition monolithic.
Let us (re-)define the XGBoost model as well as a second model with a slightly different pipeline that additionally applies normalisation and compare the two models.

In [ ]:
xgb_model = sensai.xgboost.XGBGradientBoostedVectorClassificationModel() \
    .with_raw_input_transformers(
        DFTTitanicDropMetaDataColumns(),
        DFTTitanicTransformPassengerClass()) \
    .with_name("XGBoost") \
    .with_feature_collector(feature_collector, shared=True) \
    .with_feature_transformers(
        feature_collector.create_feature_transformer_one_hot_encoder(ignore_unknown=True))

torch_mlp_model = sensai.torch.models.MultiLayerPerceptronVectorClassificationModel(
        hid_activation_function=torch.relu,
        hidden_dims=[10, 10, 4],
        cuda=False,
        p_dropout=0.25,
        nn_optimiser_params=sensai.torch.NNOptimiserParams(early_stopping_epochs=10)) \
    .with_name("MLP") \
    .with_raw_input_transformers(
        DFTTitanicDropMetaDataColumns(),
        DFTTitanicTransformPassengerClass()) \
    .with_feature_collector(feature_collector, shared=True) \
    .with_feature_transformers(
        sensai.data_transformation.DFTColumnFilter(drop=[dataset.COL_PORT_EMBARKED, dataset.COL_AGE_YEARS]),
        feature_collector.create_feature_transformer_one_hot_encoder(ignore_unknown=True),
        feature_collector.create_dft_normalisation())

titanic_evaluation.compare_models([xgb_model, torch_mlp_model]).results_df

Notice that the model definitions are purely declarative: We define each model and the respective feature pipeline by injecting appropriate pipeline components.

For the multi-layer perceptron model, we notably added some additional feature transformers:
* Since this type model cannot cope with missing feature values, we added a component that drops the age and port columns, which are sometimes undefined.
* Since neural networks work best with normalised feature representations, we added the normalisation component, which uses a standard scaler (as defined in the feature generator).
